In [22]:
import numpy as np
import pandas as pd

import seaborn as sns
from scipy.stats import norm

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [23]:
iris = sns.load_dataset("iris")
iris = iris.rename(index=str, columns={'sepal_length':'1_sepal_length','sepal_width':'2_sepal_width', 'petal_length':'3_petal_length', 'petal_width':'4_petal_width'})
print('Done')

Done


Lấy dữ liệu thủ công với mỗi loại hoa lấy 80% là tập training, 20% là tập test:

In [24]:
df1 = iris[["1_sepal_length", "2_sepal_width", "3_petal_length", "4_petal_width", "species"]]
X = df1[["1_sepal_length", "2_sepal_width", "3_petal_length", "4_petal_width"]]
y = df1['species']
df_test = pd.concat([df1[40:50], df1[90:100], df1[140:150]])
X_train = pd.concat([df1[0:40], df1[50:90], df1[100:140]]).drop(['species'], axis = 1)
X_test = pd.concat([df1[40:50], df1[90:100], df1[140:150]]).drop(['species'], axis = 1)


y_labels = df1.iloc[:, 4].replace({'setosa':0, 'versicolor':1, 'virginica':2}).copy()
y_train = pd.concat([y_labels[0:40], y_labels[50:90], y_labels[100:140]])
y_test = pd.concat([y_labels[40:50], y_labels[90:100], y_labels[140:150]])
print('Done')


Done


/tmp/ipykernel_32394/2048182279.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_labels = df1.iloc[:, 4].replace({'setosa':0, 'versicolor':1, 'virginica':2}).copy()


Hàm thực hiện thuật toán NB:

Ta đã xác định được rằng các trường trong tập dữ liệu là đại lượng ngẫu nhiên liên tục, với giả thiết các trường tuân theo phân phối chuẩn và các trường độc lập với nhau, ta có công thức tính score như ở dưới

In [25]:
def predict_NB_gaussian_class(X, mu_list, std_list, pi_list):
  scores_list = []
  classes = len(mu_list)
  for c in range(classes):
    score = norm.pdf(x = X[0], loc = mu_list[c][0][0], scale = std_list[c][0][0]) * norm.pdf(x = X[1], loc = mu_list[c][0][1], scale = std_list[c][0][1]) * norm.pdf(x = X[2], loc = mu_list[c][0][2], scale = std_list[c][0][2]) * norm.pdf(x = X[3], loc = mu_list[c][0][3], scale = std_list[c][0][3]) * pi_list[c]
    scores_list.append(score)
  
  return np.argmax(scores_list)
print('Done')

Done


Thực hiện việc lấy giá trị trung bình, độ lệch chuẩn, xác suất của từng lớp pi

In [26]:
mu_list = np.split(df1.groupby('species').mean().values,[1,2])
std_list = np.split(df1.groupby('species').std().values,[1,2], axis = 0)
pi_list = df1.iloc[:,4].value_counts().values / len(df1)
print(mu_list)
print('Done')

[array([[5.006, 3.428, 1.462, 0.246]]), array([[5.936, 2.77 , 4.26 , 1.326]]), array([[6.588, 2.974, 5.552, 2.026]])]
Done


Thực hiện thuật toán NB trên tập test

In [27]:
y_pred = np.zeros(X_test.shape[0])
for i in range(X_test.shape[0]):
  y_pred[i] = predict_NB_gaussian_class(X_test.iloc[i, :], mu_list, std_list, pi_list)

  

/tmp/ipykernel_32394/3962004731.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  score = norm.pdf(x = X[0], loc = mu_list[c][0][0], scale = std_list[c][0][0]) * norm.pdf(x = X[1], loc = mu_list[c][0][1], scale = std_list[c][0][1]) * norm.pdf(x = X[2], loc = mu_list[c][0][2], scale = std_list[c][0][2]) * norm.pdf(x = X[3], loc = mu_list[c][0][3], scale = std_list[c][0][3]) * pi_list[c]


Tính accuracy giữa kết quả dự đoán và kết quả thực tế

In [29]:
print('Accuracy: ', accuracy_score(y_pred, y_test))
print('Recall :', recall_score(y_pred, y_test, average='macro'))
print('Precision: ', precision_score(y_pred, y_test, average='macro'))
print('F1 Score: ', f1_score(y_pred, y_test, average='macro'))

Accuracy:  1.0
Recall : 1.0
Precision:  1.0
F1 Score:  1.0


# Sử dụng thư viện sklearn

Phân chia dữ liệu 80/20

In [41]:
X_train_lib, X_test_lib, y_train_lib, y_test_lib = train_test_split(X, y, train_size=0.8,)

Thực hiện phân loại với thuật toán NB thông qua model GaussianNB() của thư viện sklearn

In [42]:
model = GaussianNB()
model.fit(X_train_lib, y_train_lib)
y_pred = model.predict(X_test_lib)
print('Accuracy: ', accuracy_score(y_pred, y_test_lib))
print('Recall :', recall_score(y_pred, y_test_lib, average='macro'))
print('Precision: ', precision_score(y_pred, y_test_lib, average='macro'))
print('F1 Score: ', f1_score(y_pred, y_test_lib, average='macro'))

Accuracy:  1.0
Recall : 1.0
Precision:  1.0
F1 Score:  1.0
